# 02_NYC_data_generator

## Instructions:

**To use this notebook in its current state:**

1. **Re-run the 01_EDA_clean_NYC_data notebook** to ensure you have the latest version of the cleaned dataset


2. **Set the `predict_interval` value in the "Read Dataset" section below** to indicate what year snapshot you want to predict (set to `None` if you want to include all cumulative changes, irregardless of observation year).


3. The resulting features data frame will be saved to the `../data/interim directory/`

## Todo:

The notebook works in its current form, however there are still some additional tasks I need to perform in order to clean up this code so that it can be changed into a standalong `.py` script.

I will also be adding docstrings to clarify function parameters, outputs, and functionality

An updated version will be pushed to the GitHub repo once it is complete

### Inputs:

**`../data/interim/Capital_Projects_clean.csv`**

A cleansed version of the Capital_Projects.csv file, wherein each record represents one project change.

### Outputs:

**`../data/interim/NYC_capital_projects_{predict_interval}yr.csv`**

**or if** `predict_interval=None`:

**`../data/interim/NYC_capital_projects_all.csv`** 


<a name='index'></a>

## Notebook Index

1. <a href=#imports>Imports</a>


2. <a href=#read>Read Dataset</a>


3. <a href=#functions>Define data generator functions and default parameters</a>


4. <a href=#run>Generate features dataframe</a>


5. <a href=#save>Save Resulting Dataframe</a>

<a name='imports'></a>

## Imports
Imports for function used in this notebook.

<a href=#index>index</a>

In [1]:
import os
import math
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# Avoid scientific notation output in Pandas
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.float_format = '{:,.2f}'.format
import logging

# Improve resolution of output graphcis
%config InlineBackend.figure_format ='retina'

<a name='read'></a>
## Read Dataset
Read the dataset and perform basic manipulation of headers and some fields including formatting.

<a href=#index>index</a>

In [2]:
predict_interval = 3 # number of years, None if all changes

file_path = '../data/interim/Capital_Projects_clean.csv'

if predict_interval:
    save_path = '../data/interim/NYC_capital_projects_{}yr.csv'.format(predict_interval)
else:
    save_path = '../data/interim/NYC_capital_projects_all.csv'
    
if os.path.isfile(file_path):
    print("OK - path points to file.")
else:
    print("ERROR - check the 'file_path' and ensure it points to the source file.")
    
print('\nThe resulting saved file will be saved as:\n\n{}'.format(save_path))

OK - path points to file.

The resulting saved file will be saved as:

../data/interim/NYC_capital_projects_3yr.csv


In [3]:
data = pd.read_csv(file_path)

In [4]:
# entries
print(f"Number of dataset records: {len(data)}")

# num projects
print(f"Number of unique projects in dataset: {len(data['PID'].unique())}")

Number of dataset records: 2095
Number of unique projects in dataset: 355


In [5]:
datetime_cols = [
    'Date_Reported_As_Of',
    'Design_Start',
    'Original_Schedule',
    'Forecast_Completion'
]

for col in datetime_cols:
    data[col] = pd.to_datetime(data[col])
    
# make sure data is sorted properly
data = data.sort_values(by=['PID', 'PID_Index'])

In [6]:
data.info()
data.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2095 entries, 0 to 2094
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Record_ID                2095 non-null   object        
 1   Date_Reported_As_Of      2095 non-null   datetime64[ns]
 2   PID                      2095 non-null   int64         
 3   Project_Name             2095 non-null   object        
 4   Description              2095 non-null   object        
 5   Category                 2095 non-null   object        
 6   Borough                  2095 non-null   object        
 7   Managing_Agency          2095 non-null   object        
 8   Client_Agency            2095 non-null   object        
 9   Current_Phase            2095 non-null   object        
 10  Design_Start             2095 non-null   datetime64[ns]
 11  Original_Budget          2095 non-null   float64       
 12  Budget_Forecast          2095 non-

,Record_ID,Date_Reported_As_Of,PID,Project_Name,Description,Category,Borough,Managing_Agency,Client_Agency,Current_Phase,...,Total_Budget_Changes,Original_Schedule,Forecast_Completion,Latest_Schedule_Changes,Total_Schedule_Changes,PID_Index,Change_Years,Change_Year,Current_Project_Years,Current_Project_Year
0,3-0,2014-05-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,2-Design,...,"-4,318,643.37",2020-01-13,2020-01-14,1.00,270.00,0,0.60,1,5.94,6
1,3-1,2015-02-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,3-Construction Procurement,...,"-4,318,643.37",2020-01-13,2020-07-19,187.00,270.00,1,1.36,2,5.94,6
2,3-2,2015-08-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,3-Construction Procurement,...,"-4,318,643.37",2020-01-13,2020-08-08,20.00,270.00,2,1.85,2,5.94,6


<a name='functions'></a>
# Define data generator functions and default parameters

<a href=#index>index</a>

In [7]:
endstate_columns = [
    'Date_Reported_As_Of',
    'Change_Years',
    'PID',
    'Current_Phase',
    'Budget_Forecast',
    'Forecast_Completion',
    'PID_Index',
]

endstate_column_rename_dict = {
    'Date_Reported_As_Of': 'Final_Change_Date',
    'Current_Phase': 'Phase_End',
    'Budget_Forecast': 'Budget_End',
    'Forecast_Completion': 'Schedule_End',
    'PID_Index': 'Number_Changes',
    'Change_Years': 'Final_Change_Years'
}

info_columns = [
    'PID',
    'Project_Name',
    'Description',
    'Category',
    'Borough',
    'Managing_Agency',
    'Client_Agency',
    'Current_Phase',
    'Current_Project_Years',
    'Current_Project_Year',
    'Design_Start',
    'Original_Budget',
    'Original_Schedule',
]

info_column_rename_dict = {
    'Current_Phase': 'Phase_Start',
    'Original_Budget': 'Budget_Start',
    'Original_Schedule': 'Schedule_Start',
}


In [8]:
def extract_project_details(df, copy_columns=info_columns,
                            column_rename_dict=info_column_rename_dict,
                            use_record=0, record_index='PID_Index'):
    """
    """
    df_details = df.copy().loc[df[record_index]==use_record][copy_columns]
    
    if column_rename_dict:
        df_details = df_details.copy().rename(columns=column_rename_dict) 
    
    return df_details.set_index('PID')


def subset_project_changes(df, change_year_interval=3, change_col='Change_Year',
                           project_age_col='Current_Project_Year', inclusive_stop=True):
    """
    """
    df_subset = df.copy().loc[
        (df[change_col]<=change_year_interval) & (
            df[project_age_col]>=change_year_interval if inclusive_stop
            else df[project_age_col]>change_year_interval
        )
    ]
    
    return df_subset.set_index('Record_ID')


def find_max_record_indices(df, record_index='PID_Index'):
    """
    """
    df_group = df.groupby('PID').agg({record_index: max})
    pid_dict = dict(zip(df_group.index, df_group.values.ravel()))
    record_id_indices = [
        str(pid) + '-' + str(pid_index)
        for pid, pid_index in pid_dict.items()
    ]
    
    return record_id_indices


def project_interval_endstate(df, keep_columns=endstate_columns,
                              column_rename_dict=endstate_column_rename_dict,
                              change_year_interval=None, record_index='PID_Index',
                              change_col='Change_Year', project_age_col='Current_Project_Year',
                              inclusive_stop=True):
    """
    """
    if change_year_interval:
        df = subset_project_changes(
            df.copy(), change_year_interval, change_col, project_age_col, inclusive_stop)
    else:
        df = df.copy().set_index('Record_ID')
    
    max_record_list = find_max_record_indices(df, record_index)
    
    df_endstate = df.copy().loc[max_record_list][keep_columns]
    
    if column_rename_dict:
            df_endstate = df_endstate.copy().rename(columns=column_rename_dict)
    
    return df_endstate.set_index('PID')


def join_data_endstate(df_details, df_endstate, how='inner'):
    """
    """
    df_join = pd.merge(df_details, df_endstate, how=how, left_index=True, right_index=True)
    
    return df_join.reset_index()


def add_change_features(df):
    """
    """
    # copy input for comparison of outputs
    df_copy = df.copy()

    # calculate interval change features
    df_copy['Duration_Start'] = (df_copy['Schedule_Start'] - df_copy['Design_Start']).dt.days
    df_copy['Duration_End'] = (df_copy['Schedule_End'] - df_copy['Design_Start']).dt.days
    df_copy['Schedule_Change'] = df_copy['Duration_End'] - df_copy['Duration_Start']
    df_copy['Budget_Change'] = df_copy['Budget_End'] - df_copy['Budget_Start']

    # define schedule change ratio
    df_copy['Schedule_Change_Ratio'] = df_copy['Schedule_Change']/df_copy['Duration_Start']
    # define budget change ratio
    df_copy['Budget_Change_Ratio'] = df_copy['Budget_Change']/df_copy['Budget_Start']
    
    # define project metrics
    df_copy['Budget_Abs_Per_Error'] = (
        df_copy['Budget_Start'] - df_copy['Budget_End']
    ).abs() / df_copy['Budget_End']
    
    df_copy['Budget_Rel_Per_Error'] = (
        df_copy['Budget_Start'] - df_copy['Budget_End']
    ).abs() / df_copy['Budget_Start']
   
    df_copy['Duration_End_Ratio'] = df_copy['Duration_End']/df_copy['Duration_Start']
    df_copy['Budget_End_Ratio'] = df_copy['Budget_End']/df_copy['Budget_Start']

    # previously titled 'Mark Metric'
    df_copy['Duration_Ratio_Inv'] = (df_copy['Duration_Start']/df_copy['Duration_End']) - 1
    df_copy['Budget_Ratio_Inv'] = (df_copy['Budget_Start']/df_copy['Budget_End']) - 1
    
    return df_copy


<a name='run'></a>

# Generate Features Dataframe

<a href=#index>index</a>

In [10]:
df_endstate = project_interval_endstate(data, change_year_interval=predict_interval)

df_endstate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 3 to 1004
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Final_Change_Date   149 non-null    datetime64[ns]
 1   Final_Change_Years  149 non-null    float64       
 2   Phase_End           149 non-null    object        
 3   Budget_End          149 non-null    float64       
 4   Schedule_End        149 non-null    datetime64[ns]
 5   Number_Changes      149 non-null    int64         
dtypes: datetime64[ns](2), float64(2), int64(1), object(1)
memory usage: 8.1+ KB


In [11]:
df_details = extract_project_details(data)

df_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355 entries, 3 to 1014
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Project_Name           355 non-null    object        
 1   Description            355 non-null    object        
 2   Category               355 non-null    object        
 3   Borough                355 non-null    object        
 4   Managing_Agency        355 non-null    object        
 5   Client_Agency          355 non-null    object        
 6   Phase_Start            355 non-null    object        
 7   Current_Project_Years  355 non-null    float64       
 8   Current_Project_Year   355 non-null    int64         
 9   Design_Start           355 non-null    datetime64[ns]
 10  Budget_Start           355 non-null    float64       
 11  Schedule_Start         355 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(7)
memory us

In [12]:
# Merge PID details and PID endstate dataframes

df_merged = join_data_endstate(df_details, df_endstate)

df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PID                    149 non-null    int64         
 1   Project_Name           149 non-null    object        
 2   Description            149 non-null    object        
 3   Category               149 non-null    object        
 4   Borough                149 non-null    object        
 5   Managing_Agency        149 non-null    object        
 6   Client_Agency          149 non-null    object        
 7   Phase_Start            149 non-null    object        
 8   Current_Project_Years  149 non-null    float64       
 9   Current_Project_Year   149 non-null    int64         
 10  Design_Start           149 non-null    datetime64[ns]
 11  Budget_Start           149 non-null    float64       
 12  Schedule_Start         149 non-null    datetime64[ns]
 13  Final

In [13]:
# add additional features to dataframe

df_features = add_change_features(df_merged)
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PID                    149 non-null    int64         
 1   Project_Name           149 non-null    object        
 2   Description            149 non-null    object        
 3   Category               149 non-null    object        
 4   Borough                149 non-null    object        
 5   Managing_Agency        149 non-null    object        
 6   Client_Agency          149 non-null    object        
 7   Phase_Start            149 non-null    object        
 8   Current_Project_Years  149 non-null    float64       
 9   Current_Project_Year   149 non-null    int64         
 10  Design_Start           149 non-null    datetime64[ns]
 11  Budget_Start           149 non-null    float64       
 12  Schedule_Start         149 non-null    datetime64[ns]
 13  Final

# Save resulting features dataframe

### Examine relationships between metrics by project

In [154]:
print(
    'The resulting dataframes will be saved to .csv at the following path locations:\n'
)
print(save_path)

The resulting dataframes will be saved to .csv at the following path locations:

../data/interim/NYC_capital_projects_3yr.csv


In [62]:
data.to_csv(save_path_cleansed, index=False)